# FAKE REVIEW NODE EMBEDDING

In [1]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [2]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [4]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [5]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 152.9 MB 34 kB/s 


In [6]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

Using backend: pytorch


In [7]:
dgl.__version__

'0.7.2'

In [8]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [10]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [11]:
#loading dataset
df=pd.read_csv('/content/deceptive.csv')
df.head()

,deceptive,hotel,polarity,source,text,Class
0,truthful,conrad,positive,TripAdvisor,"We stayed for a one night getaway with family on a thursday. Triple AAA rate of 173 was a steal. 7th floor room complete with 44in plasma TV bose stereo, voss and evian water, and gorgeous bathroom(no tub but was fine for us) Concierge was very helpful. You cannot beat this location... Only flaw was breakfast was pricey and service was very very slow(2hours for four kids and four adults on a friday morning) even though there were only two other tables in the restaurant. Food was very good so it was worth the wait. I would return in a heartbeat. A gem in chicago... \n",1
1,truthful,hyatt,positive,TripAdvisor,"Triple A rate with upgrade to view room was less than $200 which also included breakfast vouchers. Had a great view of river, lake, Wrigley Bldg. & Tribune Bldg. Most major restaurants, Shopping, Sightseeing attractions within walking distance. Large room with a very comfortable bed. \n",1
2,truthful,hyatt,positive,TripAdvisor,"This comes a little late as I'm finally catching up on my reviews from the past several months:) A dear friend and I stayed at the Hyatt Regency in late October 2007 for one night while visiting a friend and her husband from out of town. This hotel is perfect, IMO. Easy check in and check out. Lovely, clean, comfortable rooms with great views of the city. I know this area pretty well and it's very convenient to many downtown Chicago attractions. We had dinner and went clubing with our friends around Division St.. We had no problems getting cabs back and forth to the Hyatt and there's even public transportation right near by but we didn't bother since we only needed cabs from and to the hotel. Parking, as is usual for Chicago, was expensive but we were able to get our car out quickly (h...",1
3,truthful,omni,positive,TripAdvisor,"The Omni Chicago really delivers on all fronts, from the spaciousness of the rooms to the helpful staff to the prized location on Michigan Avenue. While this address in Chicago requires a high level of quality, the Omni delivers. Check in for myself and a whole group of people with me was under 3 minutes, the staff had plentiful recommendations for dining and events, and the rooms are some of the largest you'll find at this price range in Chicago. Even the 'standard' room has a separate living area and work desk. The fitness center has free weights, weight machines, and two rows of cardio equipment. I shared the room with 7 others and did not feel cramped in any way! All in all, a great property! \n",1
4,truthful,hyatt,positive,TripAdvisor,"I asked for a high floor away from the elevator and that is what I got. The room was pleasantly decorated, functional and very clean. I didn't need a whole lot of service but when I did they were pleasant and prompt. I used the fitness center which was well equipped and everything was in working order. It is in a great location at one end of the Michigan Avenue shopping district. \n",1


In [12]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [13]:
df.shape

(1600, 6)

In [14]:
df['Class'].value_counts()

0    800
1    800
Name: Class, dtype: int64

In [15]:
# dropping unnecessary columns. 
df.drop(['deceptive', 'hotel', 'polarity', 'source'], axis = 1, inplace=True)
df.head()

,text,Class
0,"This is the hotel for the discriminating business traveler. It offers spacious rooms that are elegant and refined. With a cozy area complete with chairs and fireplace, they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague. The restaurant offers just the location to impress a perspective client. And all of this elegance and refinement in the heart of downtown Chicago\n",0
1,"What a wonderful experience, super great, helpful, friendly staff. Angie at the front desk made sure my husbands birthday would be great. Our room was beautiful. Angie suggested we go to Sullivans for dinner..perfect choice, make the reservation for us.(best seat in the house and a special dessert) wow...Above and beyond what any other hotel or staff has done for us before..Looking forward to staying there again & again. THANKS AGAIN \n",1
2,"I stayed at the Talbott twice in the last few weeks. I was very pleased with every facet of my stays. The staff, from doormen to front desk, to concierge, to housekeeping, were wonderful. The rooms were nicely sized and clean, and well stocked. I will return to the Talbott anytime! \n",1
3,"My boyfriend and I were amazed by the breathtaking view of Lake Michigan! We are from Texas so the view of the city was very important to us. We had no problems at this hotel... rooms, service, location were top notch. \n",1
4,"Rude, rude, rude! We stayed here on July 4th weekend. We drove from Michigan, we called customer service to get directions from I-95 w/c is a major highway to Chicago-- calling them from a cellphone, telling them we're in a middle of a storm and that we're lost....we were put on hold, forwarded to another person who did not know how to get to the hotel and we were on hold again. Giving up we decided to just exit the highway and try our luck at a 7-11 store. When we finally arrived at the hotel at 2 am no one was there to take care of our car so we looked for parking ourselves. Checking in we were not greeted by anyone, even the receptionist did not greet us!! no welcome to the knickerbocker, nothing! The man at the reception was rude, which makes me think what kind of customer service ...",1


In [16]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [17]:
# applying that into function.
df["clean_text"] = df['text'].apply(lambda x: clean_text(x))
df.head(10)

,text,Class,clean_text
0,"This is the hotel for the discriminating business traveler. It offers spacious rooms that are elegant and refined. With a cozy area complete with chairs and fireplace, they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague. The restaurant offers just the location to impress a perspective client. And all of this elegance and refinement in the heart of downtown Chicago\n",0,This is the hotel for the discriminating business traveler It offers spacious rooms that are elegant and refined With a cozy area complete with chairs and fireplace they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague The restaurant offers just the location to impress a perspective client And all of this elegance and refinement in the heart of downtown Chicago
1,"What a wonderful experience, super great, helpful, friendly staff. Angie at the front desk made sure my husbands birthday would be great. Our room was beautiful. Angie suggested we go to Sullivans for dinner..perfect choice, make the reservation for us.(best seat in the house and a special dessert) wow...Above and beyond what any other hotel or staff has done for us before..Looking forward to staying there again & again. THANKS AGAIN \n",1,What a wonderful experience super great helpful friendly staff Angie at the front desk made sure my husbands birthday would be great Our room was beautiful Angie suggested we go to Sullivans for dinner perfect choice make the reservation for us wow Above and beyond what any other hotel or staff has done for us before Looking forward to staying there again again THANKS AGAIN
2,"I stayed at the Talbott twice in the last few weeks. I was very pleased with every facet of my stays. The staff, from doormen to front desk, to concierge, to housekeeping, were wonderful. The rooms were nicely sized and clean, and well stocked. I will return to the Talbott anytime! \n",1,I stayed at the Talbott twice in the last few weeks I was very pleased with every facet of my stays The staff from doormen to front desk to concierge to housekeeping were wonderful The rooms were nicely sized and clean and well stocked I will return to the Talbott anytime
3,"My boyfriend and I were amazed by the breathtaking view of Lake Michigan! We are from Texas so the view of the city was very important to us. We had no problems at this hotel... rooms, service, location were top notch. \n",1,My boyfriend and I were amazed by the breathtaking view of Lake Michigan We are from Texas so the view of the city was very important to us We had no problems at this hotel rooms service location were top notch
4,"Rude, rude, rude! We stayed here on July 4th weekend. We drove from Michigan, we called customer service to get directions from I-95 w/c is a major highway to Chicago-- calling them from a cellphone, telling them we're in a middle of a storm and that we're lost....we were put on hold, forwarded to another person who did not know how to get to the hotel and we were on hold again. Giving up we decided to just exit the highway and try our luck at a 7-11 store. When we finally arrived at the hotel at 2 am no one was there to take care of our car so we looked for parking ourselves. Checking in we were not greeted by anyone, even the receptionist did not greet us!! no welcome to the knickerbocker, nothing! The man at the reception was rude, which makes me think what kind of customer service ...",1,Rude rude rude We stayed here on July th weekend We drove from Michigan we called customer service to get directions from I w c is a major highway to Chicago calling them from a cellphone telling them we re in a middle of a storm and that we re lost we were put on hold forwarded to another person who did not know how to get to the hotel and we were on hold again Giving up we decided to just exit the highway and try our luck at a store When we finally arrived at the hotel at am no one was t

In [18]:
# dropping the uncleaned text column "text"
df = df.drop(['text'], axis=1)

In [19]:
# renaming the column
df.rename(columns = {'clean_text':'text'}, inplace = True)
df.head()

,Class,text
0,0,This is the hotel for the discriminating business traveler It offers spacious rooms that are elegant and refined With a cozy area complete with chairs and fireplace they are the perfect place to entertain clients or discuss strategy for a business meeting with a colleague The restaurant offers just the location to impress a perspective client And all of this elegance and refinement in the heart of downtown Chicago
1,1,What a wonderful experience super great helpful friendly staff Angie at the front desk made sure my husbands birthday would be great Our room was beautiful Angie suggested we go to Sullivans for dinner perfect choice make the reservation for us wow Above and beyond what any other hotel or staff has done for us before Looking forward to staying there again again THANKS AGAIN
2,1,I stayed at the Talbott twice in the last few weeks I was very pleased with every facet of my stays The staff from doormen to front desk to concierge to housekeeping were wonderful The rooms were nicely sized and clean and well stocked I will return to the Talbott anytime
3,1,My boyfriend and I were amazed by the breathtaking view of Lake Michigan We are from Texas so the view of the city was very important to us We had no problems at this hotel rooms service location were top notch
4,1,Rude rude rude We stayed here on July th weekend We drove from Michigan we called customer service to get directions from I w c is a major highway to Chicago calling them from a cellphone telling them we re in a middle of a storm and that we re lost we were put on hold forwarded to another person who did not know how to get to the hotel and we were on hold again Giving up we decided to just exit the highway and try our luck at a store When we finally arrived at the hotel at am no one was there to take care of our car so we looked for parking ourselves Checking in we were not greeted by anyone even the receptionist did not greet us no welcome to the knickerbocker nothing The man at the reception was rude which makes me think what kind of customer service training this hotel has The guy ...


In [20]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'will', 'thereby', 'whereby', 'perhaps', 'take', 'while', 'neither', 'hereupon', "'s", 'whose', 'whither', 'ever', 'give', 'twelve', 'none', 'such', 'n’t', 'yourselves', 'back', 'along', 'everything', 'were', 'somehow', 'one', 'five', 'therein', 'whom', 'whether', 'else', 'as', 'once', 'hundred', '‘ve', 'each', 'among', 'between', 'during', 'two', 'whatever', 'herself', 'top', 'all', 'or', 'say', 'did', 'next', 'down', 'quite', "'m", 'i', 'myself', 'whereupon', 'full', 'except', 'before', 'name', 'call', 'however', 'rather', 'thereupon', 'very', '‘s', 'to', 'three', 'how', '’ve', 'show', 'against', 'which', 'seem', 'he', 'until', 'it', 'in', 'ours', 'through', 'fifty', 'via', 'toward', 'are', 'have', 'of', 'be', 'empty', 'anywhere', 'amount', 'eleven', 'over', 'here', 'may', 'was', 'herein', 'where', 'my', 'beforehand', 'indeed', 'enough', 'onto', 'me', 'sometimes', 'do', 'four', 'across', 'using', 'towards', 'afterwards', 'anyone', 'every', 'n‘t', 'otherwise', '‘re', 'side', 'former'

In [21]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [22]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [23]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [24]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,0,hotel discriminating business traveler offer spacious room elegant refined cozy area complete chair fireplace perfect place entertain client discus strategy business meeting colleague restaurant offer location impress perspective client elegance refinement heart downtown chicago
1,1,wonderful experience super great helpful friendly staff angie desk sure husband birthday great room beautiful angie suggested sullivan dinner perfect choice reservation u wow hotel staff u looking forward staying thanks
2,1,stayed talbott twice week pleased facet stay staff doorman desk concierge housekeeping wonderful room nicely sized clean stocked return talbott anytime
3,1,boyfriend amazed breathtaking view lake michigan texas view city important u problem hotel room service location notch
4,1,rude rude rude stayed july weekend drove michigan called customer service direction w c major highway chicago calling cellphone telling middle storm lost hold forwarded person know hotel hold giving decided exit highway try luck store finally arrived hotel care car looked parking checking greeted receptionist greet u welcome knickerbocker man reception rude think kind customer service training hotel guy find reservation frustrated checking able u direction maybe hour earlier room spectacular small probably little bigger walk closet risk


### FEATURE EXTRACTION

In [25]:
x = df['text']
y = df['Class']

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [27]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [28]:
len(feature_names)

7905

In [29]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aaa,aaahed,aback,abassador,abd,ability,able,abound,abrupt,abruptly,...,yucky,yummo,yummy,yunan,yup,zagat,zest,zipped,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.080368,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [30]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 1000)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [31]:
chi_support = chi2_features.get_support()

In [32]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [33]:
chi_feature

['accept',
 'accepted',
 'accomidations',
 'accommodated',
 'accommodation',
 'accomodations',
 'according',
 'accurate',
 'acknowledged',
 'acted',
 'adding',
 'address',
 'adult',
 'advantage',
 'advertises',
 'advisor',
 'affair',
 'affinia',
 'affordability',
 'agent',
 'al',
 'alive',
 'allegro',
 'allergenic',
 'allergy',
 'alley',
 'allowed',
 'amazing',
 'ambassador',
 'amenity',
 'angie',
 'angry',
 'anniversary',
 'annoying',
 'anthony',
 'anybody',
 'appointed',
 'april',
 'architectes',
 'arranging',
 'ask',
 'asleep',
 'associate',
 'assuming',
 'atmosphere',
 'attached',
 'attack',
 'attempted',
 'attended',
 'attending',
 'attraction',
 'attractive',
 'aug',
 'authorization',
 'av',
 'available',
 'ave',
 'aveda',
 'avenue',
 'award',
 'aware',
 'awful',
 'bacon',
 'badly',
 'baggage',
 'bakery',
 'ball',
 'bar',
 'bare',
 'bargain',
 'barking',
 'based',
 'basement',
 'basil',
 'bath',
 'bathroom',
 'battery',
 'beautiful',
 'beautifull',
 'beautifully',
 'beauty',
 'be

In [34]:
X_kbest_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,accept,accepted,accomidations,accommodated,accommodation,accomodations,according,accurate,acknowledged,acted,...,wouldnt,wrigley,wrong,wrote,year,yellow,yes,young,yummy,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
from scipy.spatial.distance import cdist
s = cdist(u3, u3, 'euclid')
s

array([[0.        , 0.85532228, 0.57571336, ..., 0.57029375, 0.72907539,
        0.78534501],
       [0.85532228, 0.        , 0.79945054, ..., 0.78029078, 0.91785104,
        0.94819645],
       [0.57571336, 0.79945054, 0.        , ..., 0.49551653, 0.66464116,
        0.72875854],
       ...,
       [0.57029375, 0.78029078, 0.49551653, ..., 0.        , 0.66206283,
        0.71698019],
       [0.72907539, 0.91785104, 0.66464116, ..., 0.66206283, 0.        ,
        0.80823006],
       [0.78534501, 0.94819645, 0.72875854, ..., 0.71698019, 0.80823006,
        0.        ]])

In [37]:
X1=pd.DataFrame(s)
X1

,0,1,2,3,4,5,6,7,8,9,...,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599
0,0.000000,0.855322,0.575713,0.592870,0.805142,0.654766,0.706204,0.941863,0.834563,0.737170,...,0.632420,0.700583,0.817232,0.839577,0.716237,0.710001,0.743755,0.570294,0.729075,0.785345
1,0.855322,0.000000,0.799451,0.823093,0.978628,0.827140,0.888118,1.015847,0.933405,0.925033,...,0.847504,0.918560,0.994464,0.985155,0.853045,0.886439,0.900055,0.780291,0.917851,0.948196
2,0.575713,0.799451,0.000000,0.529184,0.749072,0.590473,0.643030,0.909107,0.779301,0.672506,...,0.604097,0.684730,0.709345,0.787845,0.649770,0.666496,0.678307,0.495517,0.664641,0.728759
3,0.592870,0.823093,0.529184,0.000000,0.739494,0.583599,0.634275,0.906197,0.796309,0.704700,...,0.623250,0.706028,0.773221,0.805886,0.666301,0.686133,0.706809,0.520964,0.694360,0.752084
4,0.805142,0.978628,0.749072,0.739494,0.000000,0.767731,0.805303,1.062023,0.950130,0.869844,...,0.807296,0.879913,0.930779,0.935784,0.813732,0.832749,0.855007,0.738797,0.827892,0.909610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.710001,0.886439,0.666496,0.686133,0.832749,0.689148,0.745923,0.965451,0.882489,0.803500,...,0.725387,0.793305,0.880658,0.851064,0.774762,0.000000,0.794910,0.613903,0.792607,0.810723
1596,0.743755,0.900055,0.678307,0.706809,0.855007,0.665050,0.723964,1.011140,0.866732,0.785046,...,0.714216,0.827658,0.900939,0.864037,0.757297,0.794910,0.000000,0.651663,0.748566,0.823623
1597,0.570294,0.780291,0.495517,0.520964,0.738797,0.529523,0.625217,0.889692,0.763241,0.675417,...,0.567194,0.664338,0.758806,0.746226,0.642577,0.613903,0.651663,0.000000,0.662063,0.716980
1598,0.729075,0.917851,0.664641,0.694360,0.827892,0.687659,0.672200,0.987901,0.882590,0.780653,...,0.673361,0.774450,0.891496,0.854910,0.718878,0.792607,0.748566,0.662063,0.000000,0.808230


In [38]:
# taking average
y = np.average(X1)
y

0.8168331263345656

In [39]:
adjmat_df=X1.copy()

In [40]:
adjmat_df[adjmat_df < y]=0
adjmat_df[adjmat_df >= y]=1

In [41]:
adjmat_df

,0,1,2,3,4,5,6,7,8,9,...,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1596,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1598,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
q = adjmat_df.to_numpy()
q

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [43]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(q)

In [44]:
src

array([   0,    0,    0, ..., 1599, 1599, 1599])

In [45]:
dst

array([   1,    7,    8, ..., 1592, 1593, 1596])

In [46]:
g = dgl.graph((src, dst))

In [47]:
#g.ndata
g.ndata['feat'] = torch.tensor(q)
g.ndata['feat'] 

tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [48]:
n_nodes = g.num_nodes()
n_nodes

1600

In [49]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 1597, 1598, 1599])

In [50]:
n_nodes1 = g.number_of_nodes()
n_nodes1

1600

In [51]:
n_train = int(n_nodes1 * 0.8)
n_val = int(n_nodes1 * 0.2)

In [52]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [53]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [54]:
train_mask[:n_train] = True

In [55]:
val_mask[n_train:] = True


In [56]:
g.ndata['train_mask'] = train_mask

In [57]:
g.ndata['val_mask'] = val_mask

In [58]:
s_array = df['Class'].to_numpy()
s_array

array([0, 1, 1, ..., 1, 0, 1])

In [59]:
t1 = torch.tensor(s_array)
t1

tensor([0, 1, 1,  ..., 1, 0, 1])

In [60]:
g.ndata["label"] =t1

In [61]:
label = g.ndata["label"]

In [62]:
label[train_mask]

tensor([0, 1, 1,  ..., 0, 0, 0])

In [63]:
label[val_mask]

tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,

In [64]:
g.ndata

{'feat': tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]], dtype=torch.float64), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([0, 1, 1,  ..., 1, 0, 1])}

In [65]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [66]:
val_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [67]:
g.ndata['feat'][train_mask]

tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 0., 1., 1.]], dtype=torch.float64)

In [68]:
clf = LogisticRegression()
clf.fit(g.ndata['feat'][train_mask], label[train_mask])

#y_hat_val_lr = clf.predict_proba(X_kbest_features[val_mask,:])
y_val_lr = clf.predict(g.ndata['feat'][val_mask])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [69]:
accuracy_score(label[val_mask], y_val_lr)

0.809375

In [70]:
clf_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
clf_balanced.fit(g.ndata['feat'][train_mask], label[train_mask])

y_val_lrb = clf_balanced.predict(g.ndata['feat'][val_mask])

In [71]:
accuracy_score(label[val_mask], y_val_lrb)

0.825

In [72]:
rf = RandomForestClassifier(n_estimators=400)
rf.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = rf.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)


0.80625

In [73]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = gb.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.796875

In [74]:
ab = AdaBoostClassifier(n_estimators=400, random_state=0)
ab.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = ab.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.7875

In [75]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [76]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=10000) 
scores = cross_val_score(log, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.82875


In [77]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
scores = cross_val_score(lsvm, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.798125


In [78]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
scores = cross_val_score(pa, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.81375


In [79]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.80625


In [80]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
scores = cross_val_score(xgbc, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.80375
